NPC Dialogue Generation with LLMs

Murray Shanahan

October 2024, last modified February 2025

In [38]:
import os
import sys
from iconic_tools.langchain import InstructSonnet, InstructOpus3, InstructGPT4, InstructO1, InstructGeminiPro, InstructGeminiFlash, InstructGeminiFlash2,  InstructGPT35
from langchain_core.prompts import ChatPromptTemplate

In [39]:
# CONSTANTS AND INITIALISATION

PATH = os.path.abspath(os.getcwd())

# DIALOGUE_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGeminiPro(temperature=0.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGPT4(temperature=0.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGeminiFlash(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGeminiFlash(temperature=0.0, max_tokens=3000)

DIALOGUE_MODEL = InstructGeminiFlash2(temperature=1.0, max_tokens=3000)
QUERY_MODEL = InstructGeminiFlash2(temperature=0.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructO1()
# QUERY_MODEL = InstructO1()

# DIALOGUE_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructSonnet(temperature=0.0, max_tokens=3000)

GAME = "act_1"
# ACTORS = ["Eliza", "Player"]
ACTORS = ["Conductor", "Player"]

# SCENE = "pod"
# SCENE = "engineer"
# SCENE = "fire"
# SCENE = "find_exit"
# SCENE = "space_walk"
SCENE = "conductor"

RED = "\033[91m"
GREEN = "\033[92m"
BLUE = "\033[94m"
YELLOW = "\033[93m"
WHITE = "\033[0m"

In [40]:
# UTILITIES


def print_header(model_name):
    print(WHITE + "Game or movie: {}".format(GAME))
    print("Scene name: {}".format(SCENE))
    print("Dialogue model: {}".format(model_name))
    print()


def load_prompt(filename):
    with open(PATH + f"/prompts/{filename}") as f:
        return f.read()


def write_transcript(dialogue, filename):
    with open(PATH + f"/transcripts/{filename}", "w") as f:
        f.write(dialogue)


def list_to_conjunction(L):
    """Takes a list strings and returns a string with every element in the list separated by commas."""
    if L == "":
        return ""
    elif len(L) == 1:
        return L[0]
    elif len(L) == 2:
        return f"{L[0]} and {L[1]}"
    else:
        return ", ".join(L[:-1]) + f", and {L[-1]}"


def list_to_string(L):
    """Takes a list of strings and returns a string consisting of every element in the list separated by a newline."""
    return "\n".join(L)


def split_text(text):
    # Split the text by double newlines
    paragraphs = text.split('\n\n')
    # Remove leading or trailing whitespacen  and remove empty paragraphs
    paragraphs = [p.strip() for p in paragraphs]
    paragraphs = [p for p in paragraphs if p]
    return paragraphs


def read_queries(filename):
    queries = split_text(load_prompt(filename))
    queries = [s.strip() for s in queries if not s.strip().startswith('#')]
    return queries

In [41]:
# PROMPT TEMPLATES AND INSTRUCTION PROMPTS


dialogue_instruction_prefix = load_prompt("/instruction_prefix.txt")

preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
Here is a description of the scene in question. {scene_description}{scene_supplement}
"""

instruction_template = """
{preamble}
Here is the dialogue so far:\n
{dialogue}
{instruction_suffix}
"""

speech_template = '[{actor}]: {speech}\n'

dialogue_instruction_suffix = """
Give me the next line in the dialogue in the same format. Don't provide stage directions, just the character's words. Don't give me a line for the player, but for one of the other characters.\n
"""

query_preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
"""

query_instruction_prefix = """
You are going to answer a single question about the current state of the dialogue in a scene in the middle of a computer game.
"""

query_instruction_suffix_template = """
Now consider the following statement about this dialogue. {statement} Is this statement true or false? Answer with a single word, true or false.
"""

In [42]:
# BUILDING DIALOGUES


def prompt_llm(prompt, model):
    # print(prompt)
    # print()
    prompt = ChatPromptTemplate.from_template(template=prompt)
    chain = prompt | model
    return chain


def load_prompts(supplement_version=-1):
    """Loads a set of prompts for the current game and scene.

    Args:
        supplement_version: Version no. of supplemental scene description. Used to simulate adversarial players, for example.

    Returns:
        back_story: The backdrop to the whole game.
        scene_description: A description of this particular mini-scene. Should state the goals of the scene.
        opening_speech: The first words spoken by actor 1. (These have to be scripted.)
        queries: A list of natural language questions; the scene is terminated if all answers are yes.
    """

    back_story = load_prompt(GAME + "/back_story.txt")
    scene_description = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_description" + ".txt")
    if supplement_version == -1:  # no supplementary scene text
        scene_supplement = ""
    else:
        scene_supplement = "\n\n" + load_prompt(
            GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_supplement" + ".txt")
    opening_speech = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_opening_speech.txt")
    queries = read_queries(GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_queries.txt")
    return (back_story, scene_description, scene_supplement, opening_speech, queries)


def sim_mini_scene(supplement_version, player, max_turns):
    """Generates dialogue for a mini-scene.

    Args:
        supplement_version: Version no. of supplemental scene description. Used to simulate adversarial players, for example.
        player: If True then the user is one of the players, otherwise both players are LLMs.
        max_turns: The scene will terminate when this many turns have been taken whether or not goals have been reached.

    Returns:
        dialogue: The generated dialogue as a list of strings.
        success: True if the dialogue ended before exceeding max_turns.
    """

    dialogue_model = DIALOGUE_MODEL
    actors = ACTORS   # list of the names of the actors involved in the dialogue
 
    (back_story, scene_description, scene_supplement,
     opening_speech, queries) = load_prompts(supplement_version)
            
    dialogue_preamble = preamble_template.format(
        instruction_prefix=dialogue_instruction_prefix,
        back_story=back_story,
        scene_description=scene_description,
        scene_supplement=scene_supplement,
        actors=list_to_conjunction(actors))
    
    query_preamble = preamble_template.format(
        instruction_prefix=query_instruction_prefix,
        back_story=back_story,
        scene_description="",
        scene_supplement="",
        actors=list_to_conjunction(actors))
    
    # Deliver opening speech (which can have multiple lines)
    lines = split_text(opening_speech)
    dialogue = ""
    for line in lines:
        response = speech_template.format(actor=actors[0], speech=line)
        dialogue += response + "\n"
        print(GREEN + response)

    turn = 1  # note: each time a new actor speaks is a new turn
    success = False

    while turn < max_turns and not success:
        
        if player and (turn % 2 == 1):  # user playing actor 2, and it's their turn
            speech = ""
            while speech == "":  # ignore blank responses
                speech = input()
            response = speech_template.format(actor=actors[1], speech=speech)
        else:  # LLM's turn or both actors are played by the LLM
            prompt = instruction_template.format(
                preamble=dialogue_preamble, dialogue=dialogue,
                instruction_suffix=dialogue_instruction_suffix)
            # print(WHITE + prompt)
            # print()
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({}).strip() + "\n"

        dialogue += response + "\n"
        print(GREEN + response)

        # Have the conditions for ending the scene been met?
        fails = 0
        for statement in queries:                
            instruction = query_instruction_suffix_template.format(statement=statement)
            prompt = instruction_template.format(
                preamble=query_preamble, dialogue=dialogue,
                instruction_suffix=instruction)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({})
            if response[0:4] != "True" and response[0:3] != "true":
                fails += 1
            # print(BLUE + 'Query: {}'.format(statement))
            # print('Response: {}'.format(response))
            # print()
        success = (fails == 0)
        # print(GREEN)
        # print()

        turn += 1

    if success:
        print('Mini scene completed successfully')
    else:
        print('Mini scene ended unsuccessfully')
    
    return (dialogue, success)

In [43]:
# GENERATE ONE DIALOGUE

player = True
supplement_version = -1

dialogue = sim_mini_scene(supplement_version, player=player, max_turns=25)

[Conductor]: Welcome to light rail, The Luminata's number one transport provider.

[Conductor]: We’ll get you where you want to go at lightning speed ... So for the love of god don’t do anything to disrupt your journey for you or your fellow passengers.

[Conductor]: So, where would you like to go, friend?

[Player]: Er, how about the Biome, dude?

[Conductor]: The Biome, huh? Oh jeez, I dunno friend. I really don't like going there any more. I just get this ... *feeling* you know? It's supposed to be a slittle slice of nature... a place of relaxation. But, ever since Celebration Day... well, let's just say I wouldn't want to spoil the vibe, know what I mean? How about one of the other options?


[Player]: But I really want to go to the Biome, man.

[Eliza]: Hey, listen, I know you want to see the Biome, but maybe the conductor's right... it *was* pretty bad there on Celebration Day. Let's just pick somewhere else for now, okay? We don't want to spook the poor guy more than we already 